In [1]:
%matplotlib inline
from matplotlib import pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import datetime
import pandas as pd

torch.set_printoptions(edgeitems=2)
torch.manual_seed(123)

In [2]:
class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']

In [3]:
from torchvision import datasets, transforms
data_path = '~/data/data-unversioned/p1ch7/'
cifar10 = datasets.CIFAR10(
    data_path, train=True, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

Files already downloaded and verified


In [4]:
cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

Files already downloaded and verified


In [5]:
label_map = {0: 0, 2: 1}
class_names = ['airplane', 'bird']
cifar2 = [(img, label_map[label])
          for img, label in cifar10
          if label in [0, 2]]
cifar2_val = [(img, label_map[label])
              for img, label in cifar10_val
              if label in [0, 2]]

## 1.1

In [6]:
model = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=5, padding=1),
            nn.Tanh(),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 8, kernel_size=5, padding=1),
            nn.Tanh(),
            nn.MaxPool2d(2),
            # ... <1>
            nn.Flatten(), 
            nn.Linear(8 * 8 * 8, 32),
            nn.Tanh(),
            nn.Linear(32, 2))

In [7]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=5, padding=2)
        self.act1 = nn.Tanh()
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(16, 8, kernel_size=5, padding=2)
        self.act2 = nn.Tanh()
        self.pool2 = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(8 * 8 * 8, 32)
        self.act3 = nn.Tanh()
        self.fc2 = nn.Linear(32, 2)

    def forward(self, x):
        out = self.pool1(self.act1(self.conv1(x)))
        out = self.pool2(self.act2(self.conv2(out)))
        out = out.view(-1, 8 * 8 * 8)
        out = self.act3(self.fc1(out))
        out = self.fc2(out)
        return out
    
model = Net()

👆

注意`padding`, 加边

In [8]:
numel_list = [p.numel() for p in model.parameters()]
sum(numel_list), numel_list

(20906, [1200, 16, 3200, 8, 16384, 32, 64, 2])

变大了, lol

---

## 1.2

In [9]:
device = (torch.device('cuda') if torch.cuda.is_available()
          else torch.device('cpu'))

print(f"Training on device: {device}.")

Training on device: cuda.


In [10]:
cifar2_gpu = [(
    img.to(device=device),
    torch.tensor(label).to(device=device)
    )
    for img, label in cifar2
]

cifar2_val_gpu = [(
    img.to(device=device),
    torch.tensor(label).to(device=device)
    )
    for img, label in cifar2_val
]

In [11]:
class ResBlock(nn.Module):
    def __init__(self, n_chans):
        super(ResBlock, self).__init__()
        self.conv = nn.Conv2d(n_chans, n_chans, kernel_size=3,
                              padding=1, bias=False)
        self.batch_norm = nn.BatchNorm2d(num_features=n_chans)
        torch.nn.init.kaiming_normal_(self.conv.weight,
                                      nonlinearity='relu')
        torch.nn.init.constant_(self.batch_norm.weight, 0.5)
        torch.nn.init.zeros_(self.batch_norm.bias)

    def forward(self, x):
        out = self.conv(x)
        out = self.batch_norm(out)
        out = torch.relu(out)
        return out + x

In [12]:
class NetResDeep(nn.Module):
    def __init__(self, n_chans1=32, n_blocks=10):
        super().__init__()
        self.n_chans1 = n_chans1
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1)
        self.resblocks = nn.Sequential(
            *(n_blocks * [ResBlock(n_chans=n_chans1)]))
        self.fc1 = nn.Linear(8 * 8 * n_chans1, 32)
        self.fc2 = nn.Linear(32, 2)
        
    def forward(self, x):
        out = F.max_pool2d(torch.relu(self.conv1(x)), 2)
        out = self.resblocks(out)
        out = F.max_pool2d(out, 2)
        out = out.view(-1, 8 * 8 * self.n_chans1)
        out = torch.relu(self.fc1(out))
        out = self.fc2(out)
        return out

In [13]:
def training_loop(n_epochs, optimizer, model, loss_fn,
                        train_loader, val_loader):
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        loss_val = 0.0
        for imgs, labels in train_loader:
            imgs = imgs.to(device=device)
            labels = labels.to(device=device)
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)

            l2_lambda = 0.001
            l2_norm = sum(p.pow(2.0).sum()
                          for p in model.parameters())
            loss = loss + l2_lambda * l2_norm

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            loss_train += loss.item()

        for imgs_val, labels_val in val_loader:
            imgs_val = imgs_val.to(device=device)
            labels_val = labels_val.to(device=device)
            outputs_val= model(imgs_val)
            loss_val = loss_fn(outputs_val, labels_val)
            loss_val += loss_val.item()
        if epoch == 1 or epoch % 10 == 0:
            print('{} Epoch {}, Training loss {}, Validation loss {}'.format(
                datetime.datetime.now(), epoch,
                loss_train / len(train_loader),
                loss_val / len(val_loader)))


In [14]:
def validate(model, train_loader, val_loader):
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0

        with torch.no_grad():
            for imgs, labels in loader:
                imgs = imgs.to(device=device)
                labels = labels.to(device=device)
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1)
                total += labels.shape[0]
                correct += int((predicted == labels).sum())

        print("Accuracy {}: {:.2f}".format(name , correct / total))

In [15]:
train_loader_gpu = torch.utils.data.DataLoader(cifar2_gpu, batch_size=64, shuffle=True) 
val_loader_gpu = torch.utils.data.DataLoader(cifar2_val_gpu, batch_size=64, shuffle=False)

In [16]:
model = NetResDeep(n_chans1=32).to(device=device)
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs = 100,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader_gpu,
    val_loader = val_loader_gpu
)

2024-02-03 15:04:44.100956 Epoch 1, Training loss 0.6045157738551972, Validation loss 0.026439165696501732
2024-02-03 15:05:06.040490 Epoch 10, Training loss 0.3217203842037043, Validation loss 0.029808033257722855
2024-02-03 15:05:29.643406 Epoch 20, Training loss 0.22612076300154826, Validation loss 0.040951505303382874
2024-02-03 15:05:52.589219 Epoch 30, Training loss 0.17968029799355065, Validation loss 0.03860970959067345
2024-02-03 15:06:14.296869 Epoch 40, Training loss 0.16121861822665876, Validation loss 0.06223645433783531
2024-02-03 15:06:36.534765 Epoch 50, Training loss 0.1680987681363039, Validation loss 0.018849270418286324
2024-02-03 15:07:00.352941 Epoch 60, Training loss 0.10739076061613241, Validation loss 0.07567769289016724
2024-02-03 15:07:24.547622 Epoch 70, Training loss 0.14430729907219578, Validation loss 0.04462164640426636
2024-02-03 15:07:47.818676 Epoch 80, Training loss 0.21990443462399162, Validation loss 0.041996341198682785
2024-02-03 15:08:10.858368 

In [17]:
temp = validate(model, train_loader_gpu, val_loader_gpu)

Accuracy train: 1.00
Accuracy val: 0.90


感觉`overfit`, 换单纯的`resnet`

In [18]:
class NetRes(nn.Module):
    def __init__(self, n_chans1=32):
        super().__init__()
        self.n_chans1 = n_chans1
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(n_chans1, n_chans1 // 2, kernel_size=3,
                               padding=1)
        self.conv3 = nn.Conv2d(n_chans1 // 2, n_chans1 // 2,
                               kernel_size=3, padding=1)
        self.fc1 = nn.Linear(4 * 4 * n_chans1 // 2, 32)
        self.fc2 = nn.Linear(32, 2)
        
    def forward(self, x):
        out = F.max_pool2d(torch.relu(self.conv1(x)), 2)
        out = F.max_pool2d(torch.relu(self.conv2(out)), 2)
        out1 = out
        out = F.max_pool2d(torch.relu(self.conv3(out)) + out1, 2)
        out = out.view(-1, 4 * 4 * self.n_chans1 // 2)
        out = torch.relu(self.fc1(out))
        out = self.fc2(out)
        return out

In [19]:
model = NetRes(n_chans1=32).to(device=device)
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs = 100,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader_gpu,
    val_loader = val_loader_gpu
)

2024-02-03 15:08:36.696021 Epoch 1, Training loss 0.6956133903211849, Validation loss 0.03914457559585571
2024-02-03 15:08:43.330298 Epoch 10, Training loss 0.3681899948864226, Validation loss 0.02247072197496891
2024-02-03 15:08:50.632500 Epoch 20, Training loss 0.3312135536199922, Validation loss 0.022993098944425583
2024-02-03 15:08:58.566342 Epoch 30, Training loss 0.29430191579518045, Validation loss 0.02033849246799946
2024-02-03 15:09:05.694513 Epoch 40, Training loss 0.27118006908589865, Validation loss 0.015692278742790222
2024-02-03 15:09:12.877825 Epoch 50, Training loss 0.24556320440617335, Validation loss 0.0163732897490263
2024-02-03 15:09:19.822427 Epoch 60, Training loss 0.2307935774705972, Validation loss 0.014913669787347317
2024-02-03 15:09:26.735296 Epoch 70, Training loss 0.21132808510854745, Validation loss 0.03199885040521622
2024-02-03 15:09:33.657071 Epoch 80, Training loss 0.18891713516727374, Validation loss 0.02573925256729126
2024-02-03 15:09:40.736498 Epoc

In [20]:
temp = validate(model, train_loader_gpu, val_loader_gpu)

Accuracy train: 0.96
Accuracy val: 0.91


## 1.3

In [21]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=(3, 1), padding=1)
        self.act1 = nn.Tanh()
        self.pool1 = nn.MaxPool2d(2) # 16x17
        self.conv2 = nn.Conv2d(16, 8, kernel_size=(3, 1), padding=1)
        self.act2 = nn.Tanh() # 16x18
        self.pool2 = nn.MaxPool2d(2) # 8x9
        self.fc1 = nn.Linear(8 * 9 * 8, 32)
        self.act3 = nn.Tanh()
        self.fc2 = nn.Linear(32, 2)

    def forward(self, x):
        out = self.pool1(self.act1(self.conv1(x)))
        out = self.pool2(self.act2(self.conv2(out)))
        out = out.view(-1, 8 * 8 * 8)
        out = self.act3(self.fc1(out))
        out = self.fc2(out)
        return out
    
model = Net()

In [22]:
model = NetRes(n_chans1=32).to(device=device)
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs = 100,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader_gpu,
    val_loader = val_loader_gpu
)

2024-02-03 15:09:48.833813 Epoch 1, Training loss 0.6371667764748737, Validation loss 0.03190235793590546
2024-02-03 15:09:55.171506 Epoch 10, Training loss 0.3567518128710947, Validation loss 0.02297438308596611
2024-02-03 15:10:02.277135 Epoch 20, Training loss 0.320785586811175, Validation loss 0.028694909065961838
2024-02-03 15:10:09.161040 Epoch 30, Training loss 0.296504604778472, Validation loss 0.02401675656437874
2024-02-03 15:10:16.492937 Epoch 40, Training loss 0.274524503927322, Validation loss 0.026515578851103783
2024-02-03 15:10:23.490752 Epoch 50, Training loss 0.251407992877778, Validation loss 0.03286182880401611
2024-02-03 15:10:30.576911 Epoch 60, Training loss 0.22628296446648372, Validation loss 0.030326414853334427
2024-02-03 15:10:37.723154 Epoch 70, Training loss 0.20501783642039936, Validation loss 0.024928584694862366
2024-02-03 15:10:44.758152 Epoch 80, Training loss 0.1883383798086719, Validation loss 0.03333517909049988
2024-02-03 15:10:51.992030 Epoch 90,

In [23]:
temp = validate(model, train_loader_gpu, val_loader_gpu)

Accuracy train: 0.87
Accuracy val: 0.85


## 1.4

In [24]:
# 懒得跑前面去运行了, 我直接把一个莫模型贴过来重新算

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=5, padding=2)
        self.act1 = nn.Tanh()
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(16, 8, kernel_size=5, padding=2)
        self.act2 = nn.Tanh()
        self.pool2 = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(8 * 8 * 8, 32)
        self.act3 = nn.Tanh()
        self.fc2 = nn.Linear(32, 2)

    def forward(self, x):
        out = self.pool1(self.act1(self.conv1(x)))
        out = self.pool2(self.act2(self.conv2(out)))
        out = out.view(-1, 8 * 8 * 8)
        out = self.act3(self.fc1(out))
        out = self.fc2(out)
        return out
    
model = Net()

class NetRes(nn.Module):
    def __init__(self, n_chans1=32):
        super().__init__()
        self.n_chans1 = n_chans1
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(n_chans1, n_chans1 // 2, kernel_size=3,
                               padding=1)
        self.conv3 = nn.Conv2d(n_chans1 // 2, n_chans1 // 2,
                               kernel_size=3, padding=1)
        self.fc1 = nn.Linear(4 * 4 * n_chans1 // 2, 32)
        self.fc2 = nn.Linear(32, 2)
        
    def forward(self, x):
        out = F.max_pool2d(torch.relu(self.conv1(x)), 2)
        out = F.max_pool2d(torch.relu(self.conv2(out)), 2)
        out1 = out
        out = F.max_pool2d(torch.relu(self.conv3(out)) + out1, 2)
        out = out.view(-1, 4 * 4 * self.n_chans1 // 2)
        out = torch.relu(self.fc1(out))
        out = self.fc2(out)
        return out
    
model = NetRes(n_chans1=32).to(device=device)
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs = 50,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader_gpu,
    val_loader = val_loader_gpu
)

temp = validate(model, train_loader_gpu, val_loader_gpu)

2024-02-03 15:11:00.184206 Epoch 1, Training loss 0.6987715762132293, Validation loss 0.03862248733639717
2024-02-03 15:11:06.627197 Epoch 10, Training loss 0.36193499899214243, Validation loss 0.01944916322827339
2024-02-03 15:11:13.790674 Epoch 20, Training loss 0.31923115433781013, Validation loss 0.015802867710590363
2024-02-03 15:11:21.068241 Epoch 30, Training loss 0.2938360104894942, Validation loss 0.0265179593116045
2024-02-03 15:11:28.296698 Epoch 40, Training loss 0.2688185013593382, Validation loss 0.014441338367760181
2024-02-03 15:11:35.463437 Epoch 50, Training loss 0.2467465505098841, Validation loss 0.01808396726846695
Accuracy train: 0.91
Accuracy val: 0.88


In [25]:
cifar2_outside = [(
    img.to(device=device),
    torch.tensor(label).to(device=device)
    )
    for img, label in cifar10
    if label not in [0, 2]]

In [26]:
num_wrong = 0

for i, (img, label) in enumerate(cifar2_outside):
    if i+1 == 1:
        print("progression: 0 %")
    elif (i+1) % 5000 == 0:
        print("progression: {} %".format((i+1)/len(cifar2_outside) * 100))
    img = img.unsqueeze(0).to(device=device)
    m = nn.Softmax(dim=1)
    prediction = m(model(img))

    if prediction[0][0] > 0.95 or prediction[0][1] < 0.05:
        # print(i, label, prediction)
        num_wrong += 1
        prediction_pd = pd.DataFrame([label.cpu().item(), prediction.cpu().detach().numpy()[0]])
        prediction_pd.to_csv('prediction.csv', mode='a')

progression: 0 %
progression: 12.5 %
progression: 25.0 %
progression: 37.5 %
progression: 50.0 %
progression: 62.5 %
progression: 75.0 %
progression: 87.5 %
progression: 100.0 %


In [27]:
num_wrong, num_wrong / len(cifar2_outside)

(9570, 0.23925)

---

In [28]:
class ResBlock(nn.Module):
    def __init__(self, n_chans):
        super(ResBlock, self).__init__()
        self.conv = nn.Conv2d(n_chans, n_chans, kernel_size=3,
                              padding=1, bias=False)
        self.batch_norm = nn.BatchNorm2d(num_features=n_chans)
        torch.nn.init.kaiming_normal_(self.conv.weight,
                                      nonlinearity='relu')
        torch.nn.init.constant_(self.batch_norm.weight, 0.5)
        torch.nn.init.zeros_(self.batch_norm.bias)

    def forward(self, x):
        out = self.conv(x)
        out = self.batch_norm(out)
        out = torch.relu(out)
        return out + x
    
class NetResDeep(nn.Module):
    def __init__(self, n_chans1=32, n_blocks=10):
        super().__init__()
        self.n_chans1 = n_chans1
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1)
        self.resblocks = nn.Sequential(
            *(n_blocks * [ResBlock(n_chans=n_chans1)]))
        self.fc1 = nn.Linear(8 * 8 * n_chans1, 32)
        self.fc2 = nn.Linear(32, 2)
        
    def forward(self, x):
        out = F.max_pool2d(torch.relu(self.conv1(x)), 2)
        out = self.resblocks(out)
        out = F.max_pool2d(out, 2)
        out = out.view(-1, 8 * 8 * self.n_chans1)
        out = torch.relu(self.fc1(out))
        out = self.fc2(out)
        return out

model = NetResDeep(n_chans1=32).to(device=device)
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs = 50,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader_gpu,
    val_loader = val_loader_gpu
)

temp = validate(model, train_loader_gpu, val_loader_gpu)

print(temp)

num_wrong = 0

for i, (img, label) in enumerate(cifar2_outside):
    if i+1 == 1:
        print("progression: 0 %")
    elif (i+1) % 5000 == 0:
        print("progression: {} %".format((i+1)/len(cifar2_outside) * 100))
    img = img.unsqueeze(0).to(device=device)
    m = nn.Softmax(dim=1)
    prediction = m(model(img))

    if prediction[0][0] > 0.95 or prediction[0][1] < 0.05:
        # print(i, label, prediction)
        num_wrong += 1
        prediction_pd = pd.DataFrame([[label.cpu().item(), prediction.cpu().detach().numpy()[0]]])
        prediction_pd.to_csv('prediction.csv', mode='a')

2024-02-03 15:12:36.817081 Epoch 1, Training loss 0.6191475906759311, Validation loss 0.032655615359544754
2024-02-03 15:12:56.036548 Epoch 10, Training loss 0.3209665701457649, Validation loss 0.019524922594428062
2024-02-03 15:13:19.558161 Epoch 20, Training loss 0.232572015968098, Validation loss 0.033993713557720184
2024-02-03 15:13:41.533700 Epoch 30, Training loss 0.17312156622576866, Validation loss 0.0372597798705101
2024-02-03 15:14:03.832739 Epoch 40, Training loss 0.11123440765840992, Validation loss 0.02317872643470764
2024-02-03 15:14:29.066409 Epoch 50, Training loss 0.1279236567058381, Validation loss 0.015563269145786762
Accuracy train: 0.99
Accuracy val: 0.90
None
progression: 0 %
progression: 12.5 %
progression: 25.0 %
progression: 37.5 %
progression: 50.0 %
progression: 62.5 %
progression: 75.0 %
progression: 87.5 %
progression: 100.0 %


In [29]:
num_wrong, num_wrong / len(cifar2_outside)

(8933, 0.223325)

In [30]:
# for i, (img, label) in enumerate(cifar2_outside):
#     outputs = model(img.unsqueeze(0))
#     _, predicted = torch.max(outputs, dim=1)
#     if predicted != label:
#         print(f"第{i}张图片被错误分类为{class_names[predicted]}, 实际上是{class_names[label]}")